In [1]:
%pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
from bs4 import BeautifulSoup

def download_file(url, download_folder):
    """Скачивает файл по указанному URL в заданную папку."""
    local_filename = url.split('/')[-1]
    # Можно добавить проверку на существование файла, чтобы не скачивать повторно
    local_path = os.path.join(download_folder, local_filename)
    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # фильтрация пустых чанков
                        f.write(chunk)
        print(f"Скачан файл: {local_filename}")
    except Exception as e:
        print(f"Ошибка при скачивании {url}: {e}")

def parse_page(page_url, download_folder):
    """Парсит страницу по заданному URL и скачивает найденные документы."""
    try:
        response = requests.get(page_url)
        response.raise_for_status()
    except Exception as e:
        print(f"Ошибка при запросе {page_url}: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    # Предполагаем, что ссылки на документы находятся в тегах <a>
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href']
        # Если ссылка относительная, добавляем домен
        if href.startswith('/'):
            href = 'https://www.hse.ru' + href
        # Фильтруем ссылки по расширениям файлов
        if any(ext in href.lower() for ext in ['.pdf', '.doc', '.docx', '.txt', '.xls', '.xlsx', '.ppt', '.pptx']):
            print(f"Найдена ссылка на файл: {href}")
            download_file(href, download_folder)


In [4]:
download_folder = "downloaded_files"
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

base_url = "https://www.hse.ru/docs/index.html"
total_pages = 108 # общее количество страниц

for page in range(1, total_pages + 1):
    # Формирование URL страницы. Если первая страница не содержит параметров,
    # то для неё используем base_url, для остальных добавляем параметр ?page=
    if page == 1:
        url = base_url
    else:
        url = f"{base_url}?page={page}"
    print(f"Обрабатывается страница: {url}")
    parse_page(url, download_folder)


Обрабатывается страница: https://www.hse.ru/docs/index.html
Найдена ссылка на файл: https://www.hse.ru/data/xf/204/967/1138/5Устав НИУ ВШЭ утвержденный постановление.. 02 2016 № 56 (рабочая версия).docx
Скачан файл: 5Устав НИУ ВШЭ утвержденный постановление.. 02 2016 № 56 (рабочая версия).docx
Найдена ссылка на файл: https://www.hse.ru/data/xf/2013/05/22/1298199474/Текст.pdf
Скачан файл: Текст.pdf
Найдена ссылка на файл: https://www.hse.ru/data/xf/555/112/1972/6.18-01_200125-1 Начальник Управления мо.. (передоверие от Ш.К.Ахметова).docx
Скачан файл: 6.18-01_200125-1 Начальник Управления мо.. (передоверие от Ш.К.Ахметова).docx
Найдена ссылка на файл: https://www.hse.ru/data/xf/236/008/1972/6.18-01_170125-9 Приложение 1.docx
Скачан файл: 6.18-01_170125-9 Приложение 1.docx
Найдена ссылка на файл: https://www.hse.ru/data/xf/585/753/1972/Положение о закупке товаров работ услуг для нужд НИУ ВШЭ 2024.docx
Скачан файл: Положение о закупке товаров работ услуг для нужд НИУ ВШЭ 2024.docx
Найдена 